In [1]:
%load_ext jupyter_black

import os
from glob import glob
import pandas as pd
from wxlab import ProbSevere, Galwem, unpack_files, BBox

In [2]:
ALL_GALWEM_FILES = sorted(glob(os.path.join("data", "galwem", "*.GR2")))
ALL_PROBSEVERE_FILES = sorted(glob(os.path.join("data", "probsevere", "*.json")))
g_files, ps_files = unpack_files(ALL_GALWEM_FILES, ALL_PROBSEVERE_FILES, time_buffer=90)

pd.concat([g_files, ps_files])

2022-05-20 00:00:00    data/galwem/PS.557WW_SC.U_DI.C_GP.GALWEM-GD_GR...
2022-05-20 03:00:00    data/galwem/PS.557WW_SC.U_DI.C_GP.GALWEM-GD_GR...
2022-05-20 06:00:00    data/galwem/PS.557WW_SC.U_DI.C_GP.GALWEM-GD_GR...
2022-05-19 22:30:39    data/probsevere/MRMS_PROBSEVERE_20220519_22303...
2022-05-19 22:32:40    data/probsevere/MRMS_PROBSEVERE_20220519_22324...
                                             ...                        
2022-05-20 07:20:38    data/probsevere/MRMS_PROBSEVERE_20220520_07203...
2022-05-20 07:22:41    data/probsevere/MRMS_PROBSEVERE_20220520_07224...
2022-05-20 07:24:35    data/probsevere/MRMS_PROBSEVERE_20220520_07243...
2022-05-20 07:26:41    data/probsevere/MRMS_PROBSEVERE_20220520_07264...
2022-05-20 07:28:35    data/probsevere/MRMS_PROBSEVERE_20220520_07283...
Length: 273, dtype: object

In [3]:
ps = ProbSevere(ps_files)
ps

ValueError: Item wrong length 25 instead of 27070.

In [ ]:
ps.to_dataframe(dtype=object).plot()

In [ ]:
ps.bounds

In [ ]:
galwem = Galwem(g_files)
galwem

In [ ]:
from geopandas import GeoDataFrame
from shapely.geometry import box


def make_bbox():
    bounds: pd.Series = (
        GeoDataFrame(geometry=[box(minx=-130.0, maxx=-60.0, miny=20.0, maxy=55.0)])
        .set_crs(epsg=4326)
        .bounds.rename({0: "bbox"})
        .squeeze()
    )
    # Longitude conversion 0~360 to -180~180
    # https://confluence.ecmwf.int/pages/viewpage.action?pageId=149337515
    bounds[["minx", "maxx"]] = bounds[["minx", "maxx"]] % 360
    return bounds


galwem_df = galwem.to_dataframe(bbox=make_bbox())
probsevere = ProbSevere(ps_files).to_grid_like(galwem_df).index_like(galwem_df.index)  # .to_dataframe(dtype=object)
# probsevere.index

In [ ]:
probsevere.index.unique("validTime")

In [ ]:
probsevere.reset_index(["lat", "lon"]).plot(kind="scatter", x="lat", y="lon")

In [ ]:
glwm_df = glwm_ds.to_dataframe()
glwm_df

In [ ]:
ps_grid = ps.to_grid_like(glwm_df)
ps_grid

In [ ]:
ps_fit = ps_grid.index_like(glwm_df.index)
ps_fit

In [ ]:
# ps_df = ps_fit.to_dataframe()
# ps_fit.frame.reset_index(["lat", "lon"]).plot(
#     kind="scatter",
#     x="lon",
#     y="lat",
# )
# ps_fit.frame["MUCAPE"].reset_index(["lat", "lon"]).plot(
#     kind="scatter",
#     x="lon",
#     y="lat",
#     alpha=0.4,
# )
ps_fit.frame

In [ ]:
# import matplotlib.pyplot as plt

# plt.rcParams["axes.grid"] = True
# fig, ax = plt.subplots()


# ps_df.reset_index(["lat", "lon"]).plot(
#     kind="scatter",
#     x="lon",
#     y="lat",
#     s="PS",
#     c="MLCAPE",
#     ax=ax,
#     figsize=(16, 12),
#     cmap=plt.get_cmap("jet"),
#     alpha=0.4,
# )